In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
solver = po.SolverFactory('glpk') # GNU Linear Programming Kit

## Model (Specific)
Consider the following binary knapsack instance.

\begin{align}
    \text{max}~~  & 3 x_1 + 4 x_2 + 5 x_3 + 8 x_4 + 9 x_5 \\
    \text{s.t.}~~ & 2 x_1 + 3 x_2 + 4 x_3 + 7 x_4 + 9 x_5 \le 20 \\
                  & x_1, x_2, x_3, x_4, x_5 \in \{0, 1\}
\end{align}

## Implement (Specific)

In [4]:
model = pe.ConcreteModel()

In [5]:
model.x1 = pe.Var(domain=pe.Binary)
model.x2 = pe.Var(domain=pe.Binary)
model.x3 = pe.Var(domain=pe.Binary)
model.x4 = pe.Var(domain=pe.Binary)
model.x5 = pe.Var(domain=pe.Binary)

In [6]:
obj_expr = 3 * model.x1 + 4 * model.x2 + 5 * model.x3 + 8 * model.x4 + 9 * model.x5
model.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [7]:
con_expr = 2 * model.x1 + 3 * model.x2 + 4 * model.x3 + 5 * model.x4 + 9 * model.x5 <= 20
model.con = pe.Constraint(expr=con_expr)

## Solve and Postprocess

In [8]:
result = solver.solve(model, tee=True)

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
print(pe.value(model.x1))
print(pe.value(model.x2))
print(pe.value(model.x3))
print(pe.value(model.x4))
print(pe.value(model.x5))
print(pe.value(model.obj))

## Model (General)
The best way to use Pyomo is to implement the general form of the problem. The instance above is generalized by the formulation below. The set of items is $N$. The coefficients $c_i$ and $a_i$ are the cost and size, respectively of item $i$.

\begin{align}
    \text{max}~~  & \sum_{i \in N}{c_i x_i} \\
    \text{s.t.}~~ & \sum_{i \in N}{a_i x_i} \le b \\
                  & x_i \in \{0, 1\}, \forall i \in N
\end{align}

## Implement (General)

In [ ]:
model = pe.ConcreteModel()

In [ ]:
model.N = pe.RangeSet(1, 5)

In [ ]:
print(set(model.N))

In [ ]:
c = {1: 3, 2: 4, 3: 5, 4: 8, 5: 9}
a = {1: 2, 2: 3, 3: 4, 4: 5, 5: 9}
b = 20

In [ ]:
model.c = pe.Param(model.N, initialize=c)
model.a = pe.Param(model.N, initialize=a)
model.b = pe.Param(initialize=b)

For most model components, `print(model.component)` will print the name of the component (a string) and not the value (in most cases, a number).

In [ ]:
print(model.c)
print(model.c[2])
print(model.b)
print(model.b.value)
print(pe.value(model.b))

In [ ]:
model.x = pe.Var(model.N, domain=pe.Binary)

In [ ]:
obj_expr = sum(model.c[i] * model.x[i] for i in model.N)
model.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [ ]:
con_lhs_expr = sum(model.a[i] * model.x[i] for i in model.N)
con_rhs_expr = model.b
model.con = pe.Constraint(expr=(con_lhs_expr <= con_rhs_expr))

In [ ]:
result = solver.solve(model)

After solving, variable values may be accessed either by `pe.value(model.myvar)` or `model.myvar.value`.

In [ ]:
for i in model.N:
    print(pe.value(model.x[i]))
print(pe.value(model.obj))